# Défi IA

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
# import eli5
# import shap
# import xgboost as xgb
# from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split 
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
#from sklearn.preprocessing import StandardScaler

In [2]:
bdd_all = pd.read_csv("pricing_requests.csv")
bdd = bdd_all[["hotel_id","price","city","date","language","mobile","avatar_id","request_number"]].copy()
bdd

,hotel_id,price,city,date,language,mobile,avatar_id,request_number
0,300,145,copenhagen,2,hungarian,0,28434,1.0
1,855,311,copenhagen,2,hungarian,0,28434,1.0
2,757,151,copenhagen,2,hungarian,0,28434,1.0
3,805,202,copenhagen,2,hungarian,0,28434,1.0
4,930,120,copenhagen,2,hungarian,0,28434,1.0
...,...,...,...,...,...,...,...,...
163000,720,280,copenhagen,31,croatian,0,76537,1.0
163001,994,164,copenhagen,31,croatian,0,76537,1.0
163002,744,173,copenhagen,31,croatian,0,76537,1.0
163003,124,277,copenhagen,31,croatian,0,76537,1.0


### On split nos données en un échantillon d'apprentissage et un échantillon test

In [3]:
bdd_dummies = pd.get_dummies(data=bdd, columns = ['hotel_id','city', 'language'], drop_first=True)
data_train,data_test = train_test_split(bdd_dummies,test_size=0.25,random_state=1)
X_train = data_train.loc[:,data_train.columns != "price"].copy()
Y_train = data_train.loc[:,"price"].copy()
X_test = data_test.loc[:,data_train.columns != "price"].copy()
Y_test = data_test.loc[:,"price"].copy()

X_all = bdd_dummies.loc[:,bdd_dummies.columns != "price"].copy()
Y_all = bdd_dummies.loc[:,"price"].copy()
# # On convertit les variables catégorielles en indicatrices 
# X_train = pd.get_dummies(data=X_train, columns = ['hotel_id','city', 'language', 'mobile','avatar_id'], drop_first=True)
# X_test = pd.get_dummies(data=X_test, columns = ['hotel_id','city', 'language', 'mobile','avatar_id'], drop_first=True)

### On entraine nos modèles et on regarde l'erreur RMSE entre Y_test et Y_pred 

In [20]:
#LinearRegression
lr = LinearRegression().fit(X_train, Y_train)
y_pred_lr = lr.predict(X_test)
RMSE_lr = math.sqrt(mean_squared_error(Y_test, y_pred_lr))
print(RMSE_lr)

5110962224.579757


In [70]:
##RandomForestRegressor
rf = RandomForestRegressor().fit(X_train, Y_train)
y_pred_rf = rf.predict(X_test)
RMSE_rf = math.sqrt(mean_squared_error(Y_test,y_pred_rf))
print(RMSE_rf)


5.537442648450712


In [5]:
##Multi-Layer Perceptron
# mlp = MLPRegressor().fit(X_train, Y_train)
y_pred_mlp = mlp.predict(X_test)
RMSE_mlp = math.sqrt(mean_squared_error(Y_test, y_pred_mlp))
print(RMSE_mlp)

24.93547053656453


In [ ]:
##GradientBoostRegressor
gbr = GradientBoostingRegressor().fit(X_train, Y_train)
y_pred_gbr = gbr.predict(X_test)
RMSE_gbr = math.sqrt(mean_squared_error(Y_test, y_pred_gbr))

In [ ]:
##XGboostRegressor
xgbr = (xgb.XGBRegressor(verbosity=0)).fit(X_train, Y_train)
y_pred_xgbr = xgbr.predict(Y_test)
RMSE_xgbr = math.sqrt(mean_squared_error(Y_test, y_pred_xgbr))

In [ ]:
#AdaBoostRegressor
abr = AdaBoostRegressor().fit(X_train, Y_train)
y_pred_abr = adr.predict(X_test)
RMSE_abr = math.sqrt(mean_squared_error(Y_test, y_pred_abr))

### On regarde l'importance de chaque variable dans chacun des modèles entraînés

In [72]:
#feature_names = ["hotel_id", "city", "date", "language", "mobile", "avatar_id", "request_number"]
feature_names = X_train.columns
print(bdd.columns)

Index(['hotel_id', 'price', 'city', 'date', 'language', 'mobile', 'avatar_id',
       'request_number'],
      dtype='object')


In [73]:
## Avec Permutation Importance 
features_importance_dict = {}
for model, name in zip([lr, rf, mlp], ['logistic regression', 'random forest', 'multi layer perceptron']):
  plt.figure()
  permutation_import = PermutationImportance(model).fit(X_test,Y_test)
  features_importance = {'Feature_name':feature_names, 'Importance':permutation_import.feature_importances_}  
  features_importance = pd.DataFrame(features_importance) # dataframe containing the features names and their importance
  features_importance = features_importance.sort_values(['Importance'], ascending = True) # sort the dataframe by feature importance
  features_importance_dict[name] = features_importance #add the dataframe to your dictionnary
  ax = sns.barplot(x="Importance", y="Feature_name", data=features_importance) #plot the model's features importance
  plt.title(name)

In [ ]:
## Avec la methode de Shapley
explainer = shap.KernelExplainer(lr.predict, X_train)
shap_values = explainer.shap_values(X_test.iloc[0:20, :])
shap.summary_plot(shap_values, X_test.iloc[0:20,:])